In [ ]:
%cd ..

In [ ]:
import json
import numpy as np
import pandas as pd
from notebooks.utils import *
pd.set_option('display.max_rows', 1000)

In [ ]:
TEX_TABLE_DIR = "../masters-thesis-tex/tables/"

MODEL_COL = "model"
METHOD_COL = "method"
DATASET_COL = "dataset"
IMAGE_COL = "image_index"
NOISE_COL = "noise_stddev"
KERNEL_COL = "kernel"
METRIC_COL = "metric"
VALUE_COL = "value"

METHOD_MAPPING = {
    "DMSP_ORIG": "DMSP \\cite{bigdeli_deep_2017}",
    "DPIR": "DPIR \\cite{zhang_plug-and-play_2021}",
    "FDN": "FDN \\cite{kruse_learning_2017}",
    "dmsp": "DMSP",
    "hqs": "HQS",
}

In [ ]:
hqs_folder = "results/nb_deblurring/hqs/"
dmsp_folder = "results/nb_deblurring/dmsp/"
dmsp_orig_csv = "results/nb_deblurring/dmsp_paper.csv"
dpir_csv = "results/nb_deblurring/dpir.csv"
fdn_csv = "results/nb_deblurring/fdn.csv"

# HQS
df_hqs = load_folder(hqs_folder, MODEL_COL)
df_hqs[METHOD_COL] = "hqs"

# DMSP
df_dmsp = load_folder(dmsp_folder, MODEL_COL)
df_dmsp[METHOD_COL] = "dmsp"

# DMSP reference code
df_dmsp_orig = pd.read_csv(dmsp_orig_csv)
df_dmsp_orig[MODEL_COL] = ""
df_dmsp_orig[METHOD_COL] = "DMSP_ORIG"

# DPIR
df_dpir = pd.read_csv(dpir_csv)
df_dpir[MODEL_COL] = ""
df_dpir[METHOD_COL] = "DPIR"

# FDN
df_fdn = pd.read_csv(fdn_csv)
df_fdn[MODEL_COL] = ""
df_fdn[METHOD_COL] = "FDN"

# Combine the datasets
df_all = pd.concat([df_hqs, df_dmsp, df_dmsp_orig, df_dpir, df_fdn], axis=0)

# Read the model name mapping
with open("models/model_name_mapping.json") as f:
    model_mapping = json.load(f)


In [ ]:
# Read the model name mapping
with open('models/model_name_mapping.json') as f:
    model_mapping = json.load(f)

model_mapping = {
    "": "",
    **model_mapping
}
models = list(model_mapping.keys())

In [ ]:
print(df_all[DATASET_COL].unique())

# Main Text Table

In [ ]:
dataset = "BSDS500"
metrics = ["PSNR", "LPIPS_ALEX"]
metrics_mapping = {
    "PSNR": "PSNR",
    "LPIPS_ALEX": "LPIPS",
}
noise_levels = [0.01, 0.02, 0.03, 0.04]
methods = ["FDN", "DMSP_ORIG", "DPIR", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]

In [ ]:
df = df_all.copy()

# Filter only the configured dataset
df = df[df[DATASET_COL] == dataset]
df = df.drop(DATASET_COL, axis=1)

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, NOISE_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, NOISE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[MODEL_COL].isin(models)]
df = df[df[NOISE_COL].isin(noise_levels)]
df = df[df[METHOD_COL].isin(methods)]

# Sort by Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(NOISE_COL, kind="mergesort")

# Remap metrics
df[METRIC_COL] = df[METRIC_COL].map(metrics_mapping)

# Noise Level and Metric as columns
df = df.set_index([METHOD_COL, MODEL_COL, NOISE_COL, METRIC_COL])
df = df.unstack([NOISE_COL, METRIC_COL])

# Sort and remap models
df = df.reset_index()
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING)
df = df.set_index([METHOD_COL, MODEL_COL])

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["Additive noise \( \sigma_n \)", ""])

# Rename model column
df.index = df.index.rename(["Method", ""])

df


## Convert to LaTeX

In [ ]:
column_format = "ll" + "C{1}" * len(df.columns)


formatters = {
    c: mark_column_best_formatter(
        df, c, mark_max=c[1] == "PSNR", num_decimals=2 if c[1] == "PSNR" else 4
    )
    for c in df.columns
}

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        # float_format=float_format(True),
        formatters=formatters,
        escape=False,
        column_format=column_format,
        multicolumn_format="c",
        multirow=True,
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Use multicolumn sometimes
latex = latex.replace(
    METHOD_MAPPING["DMSP_ORIG"] + " &",
    "\\multicolumn{2}{l}{" + METHOD_MAPPING["DMSP_ORIG"] + "}",
)
latex = latex.replace(
    METHOD_MAPPING["DPIR"] + " &",
    "\\multicolumn{2}{l}{" + METHOD_MAPPING["DPIR"] + "}",
)
latex = latex.replace("Method & {}", "\\multicolumn{2}{l}{Method}")

latex = add_midrule(latex, 9)
# latex = add_midrule(latex, 18)

# print(latex)


In [ ]:
with open(TEX_TABLE_DIR + "nb-deblurring_bsds500.tex", "w") as f:
    f.write(latex)

# Appendix Table

In [ ]:
datasets_kernels = {
    "BSDS500-schelten": "BSDS500~\cite{arbelaez_contour_2011}",
    "Set5-levin_1": "Set5 + 2nd kernel from~\cite{levin_understanding_2009}",
    "Set5-levin_3": "Set5 + 4th kernel from~\cite{levin_understanding_2009}",
}
metrics = {
    "PSNR": "P",
    "SSIM": "S",
    "FSIM": "F",
    "LPIPS_ALEX": "L",
}
noise_levels = [0.01, 0.02, 0.03, 0.04]
methods = ["FDN", "DMSP_ORIG", "DPIR", "dmsp", "hqs"]
models = [
    "",
    "dcnn_0.05",
    "drcnn_0.05",
    "dunet_0.05",
    "drunet+_0.05",
    "drunet+_0.0-0.2",
    # "drugan+-lambda-zero_0.0-0.2",
    "drugan+_0.0-0.2",
]


In [ ]:
df = df_all.copy()


# Combine dataset and kernel
df[DATASET_COL] = df[DATASET_COL] + "-" + df[KERNEL_COL]
# df = df[(df[DATASET_COL] + "-" + df[KERNEL_COL]).isin(foo)]

# Mean over the images
df = df.groupby([METHOD_COL, MODEL_COL, NOISE_COL, DATASET_COL]).mean()
df = df.drop(IMAGE_COL, axis=1)
df = df.reset_index()

# Combine all metrics in one column
df = df.melt(
    id_vars=[METHOD_COL, MODEL_COL, DATASET_COL, NOISE_COL],
    var_name=METRIC_COL,
    value_name=VALUE_COL,
)

# Filter metrics and datasets
df = df[df[METRIC_COL].isin(metrics)]
df = df[df[DATASET_COL].isin(datasets_kernels)]
df = df[df[METHOD_COL].isin(methods)]
df = df[df[MODEL_COL].isin(models)]

# Remap names
df = df.sort_values(MODEL_COL, key=sort_key_for(models), kind="mergesort")
df = df.sort_values(METHOD_COL, key=sort_key_for(methods), kind="mergesort")
df[METHOD_COL] = df[METHOD_COL].map(METHOD_MAPPING)
df[MODEL_COL] = df[MODEL_COL].map(model_mapping)

# Models as columns
df = df.set_index([DATASET_COL, NOISE_COL, METRIC_COL, METHOD_COL, MODEL_COL])
df = df.unstack([METHOD_COL, MODEL_COL])

df = df.reset_index()

# Sort by Dataset, Noise, and Metric
df = df.sort_values(METRIC_COL, key=sort_key_for(metrics), kind="mergesort")
df = df.sort_values(NOISE_COL, kind="mergesort")
df = df.sort_values(DATASET_COL, key=sort_key_for(datasets_kernels), kind="mergesort")

# Rename Dataset, Noise, and Metric
df[DATASET_COL] = df[DATASET_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{" + datasets_kernels[x] + "}"
)
df[NOISE_COL] = df[NOISE_COL].map(
    lambda x: "\rotatebox[origin=c]{90}{\( \sigma_n \)=" + str(x) + " }"
)
df[METRIC_COL] = df[METRIC_COL].map(metrics)

df = df.set_index([DATASET_COL, NOISE_COL, METRIC_COL])

# Update the column index and index
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(["", ""])
df.index = df.index.rename(["", "", ""])

# Sort models
# df = df.reindex(list(model_mapping.values()), axis=1)

df


In [ ]:
column_format = "lll|" + "C{0.7}C{0.7}C{0.7}|" + "C{1}" * 6 + "|" + "C{1}" * 2

with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(
        float_format=float_format(True),
        escape=False,
        na_rep="---",
        column_format=column_format,
        multicolumn_format="c",
        multirow=True
    )
# Use tabularx
latex = latex.replace("\\begin{tabular}", "\\begin{tabularx}{\\textwidth}")
latex = latex.replace("\\end{tabular}", "\\end{tabularx}")

# Bug in pandas?? "&" Missing in line 4
lines = latex.splitlines()
lines[3] = "& & &" + lines[3]
latex = "\n".join(lines)

latex = delete_line(latex, 4)

# print(latex)

In [ ]:
with open(TEX_TABLE_DIR + "all_nb-deblurring.tex", "w") as f:
    f.write(latex)